In [42]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *


training = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "sentimentdataset")

validation = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "validation")
testing = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "testing")

#remove nonASCIICharacters
training["Text"] = training.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
validation["Text"] = validation.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
testing["Text"] = testing.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)






In [43]:
def to_class(s):
    match s:
        case "Positive":
            return 2
        case "Negative":
            return 0
        case "Neutral":
            return 1
    

In [44]:
training["SimpleSent"] = training.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
testing["SimpleSent"] = testing.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
validation["SimpleSent"] = validation.apply(lambda x: to_class(x["SimpleSent"]), axis=1)

In [45]:
toDrop = ["Timestamp", "User", "Platform", "Retweets", "Likes", "Country", "Year", "Month", "Day", "Hour"]

training = training.drop(toDrop, axis=1)
testing = testing.drop(toDrop, axis=1)
validation = validation.drop(toDrop, axis=1)


In [47]:
import re
def split_words(xs):
    empties = []
    empty = "" 
    for i in xs:
        words = re.split(r"([A-Z])", i)
        words = words[1::]
        for i in range(0, len(words) -1, 2):
            empty += words[i] + words[i + 1]
            empty += " "
        empties.append(empty.rstrip())
    return empties

In [48]:
from textblob import *
#process Hashtages
training["proc_hash"] = training.apply(lambda x: x["Hashtags"][1::], axis=1)
training["proc_hash"] = training.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
training["proc_hash"] = training.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
training["Hashtags"] = training["proc_hash"]
training = training.drop("proc_hash", axis=1)

testing["proc_hash"] = testing.apply(lambda x: x["Hashtags"][1::], axis=1)
testing["proc_hash"] = testing.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
testing["proc_hash"] = testing.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
testing["Hashtags"] = testing["proc_hash"]
testing = testing.drop("proc_hash", axis=1)

validation["proc_hash"] = validation.apply(lambda x: x["Hashtags"][1::], axis=1)
validation["proc_hash"] = validation.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
validation["proc_hash"] = validation.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
validation["Hashtags"] = validation["proc_hash"]
validation = validation.drop("proc_hash", axis=1)

validation






,Text,Sentiment,SimpleSent,Hashtags
0,"After a series of defeats, the soccer team fac...",Disappointment,0,Disappointment and Disappointment Soccer Defeats
1,"In the tennis tournament, a highly anticipated...",Frustration,0,Frustration and Frustration Tennis Setback
2,"Facing a defeat in the championship, the boxer...",Reflection,1,Reflection and Reflection Boxing Defeat
3,"In the midst of a cycling race, a tire blowout...",Obstacle,1,Obstacle and Obstacle Cycling Frustration
4,The gymnast's unexpected fall during a routine...,Sympathy,2,Sympathy and Sympathy Gymnastics Fall
...,...,...,...,...
86,"Organized a community painting event, turning ...",Joy,2,Community Art and Community Art Senior Creativity
87,Hosted a 'memory lane' evening with old friend...,Gratitude,2,Friendship Adventures and Friendship Adventure...
88,"Joined a seniors' astronomy club, stargazing a...",Curiosity,1,Celestial Wonders and Celestial Wonders Senior...
89,"Attended a local jazz festival, tapping toes t...",Joy,2,Timeless Tunes and Timeless Tunes Senior Jazz


In [49]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["Text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["Text"]).subjectivity, axis=1)

training["hash_tb_pol"] = training.apply(lambda x: TextBlob(x["Hashtags"]).polarity, axis=1)
training["hash_tb_sub"] = training.apply(lambda x: TextBlob(x["Hashtags"]).subjectivity, axis=1)




In [50]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Text"]), axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training["text_vader_comp"] = training.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)

training["hash_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Hashtags"]), axis=1)
training["hash_vader_neg"] = training.apply(lambda x: x["hash_vader_scores"]["neg"], axis=1)
training["hash_vader_neu"] = training.apply(lambda x: x["hash_vader_scores"]["neu"], axis=1)
training["hash_vader_pos"] = training.apply(lambda x: x["hash_vader_scores"]["pos"], axis=1)
training["hash_vader_comp"] = training.apply(lambda x: x["hash_vader_scores"]["compound"], axis=1)
training = training.drop(["hash_vader_scores"], axis=1)





In [51]:
training

,Text,Sentiment,SimpleSent,Hashtags,text_tb_pol,text_tb_sub,hash_tb_pol,hash_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos,text_vader_comp,hash_vader_neg,hash_vader_neu,hash_vader_pos,hash_vader_comp
0,Enjoying a beautiful day at the park! ...,Positive,2,Nature and Nature Park,0.750000,0.800000,0.00,0.000000,0.000,0.397,0.603,0.8221,0.0,1.000,0.000,0.0000
1,Traffic was terrible this morning. ...,Negative,0,Traffic and Traffic Morning,-1.000000,1.000000,0.00,0.000000,0.437,0.563,0.000,-0.4767,0.0,1.000,0.000,0.0000
2,Just finished an amazing workout! ...,Positive,2,Fitness and Fitness Workout,0.750000,0.900000,0.00,0.000000,0.000,0.494,0.506,0.6239,0.0,0.323,0.677,0.4939
3,Excited about the upcoming weekend getaway! ...,Positive,2,Travel and Travel Adventure,0.468750,0.750000,0.00,0.000000,0.000,0.650,0.350,0.4003,0.0,0.566,0.434,0.3182
4,Trying out a new recipe for dinner tonight. ...,Neutral,1,Cooking and Cooking Food,0.136364,0.454545,0.00,0.000000,0.000,1.000,0.000,0.0000,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,2,Science Fair Winner and Science Fair Winner Hi...,0.875000,0.900000,0.52,0.780000,0.000,0.599,0.401,0.8268,0.0,0.330,0.670,0.9042
728,Attending a surprise birthday party organized ...,Happy,2,Surprise Celebration and Surprise Celebration ...,0.687500,0.600000,0.16,0.540000,0.000,0.327,0.673,0.9551,0.0,0.413,0.587,0.7269
729,Successfully fundraising for a school charity ...,Happy,2,Community Giving and Community Giving High Sch...,0.516667,0.383333,0.16,0.540000,0.000,0.468,0.532,0.9098,0.0,0.556,0.444,0.5859
730,"Participating in a multicultural festival, cel...",Happy,2,Cultural Celebration and Cultural Celebration ...,1.000000,1.000000,0.12,0.246667,0.000,0.479,0.521,0.8977,0.0,1.000,0.000,0.0000


In [57]:
'''
    Begin Neural Net
'''

classes = training["SimpleSent"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=1)
model.fit(x_train, y_train, epochs=100, verbose=2)
model.save("TWEETS_NN.keras")

Epoch 1/100
20/20 - 1s - 60ms/step - accuracy: 0.6093 - loss: 1.0546
Epoch 2/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 1.0331
Epoch 3/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 1.0111
Epoch 4/100
20/20 - 0s - 4ms/step - accuracy: 0.6029 - loss: 0.9872
Epoch 5/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 0.9622
Epoch 6/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 0.9359
Epoch 7/100
20/20 - 0s - 4ms/step - accuracy: 0.6029 - loss: 0.9065
Epoch 8/100
20/20 - 0s - 3ms/step - accuracy: 0.6045 - loss: 0.8770
Epoch 9/100
20/20 - 0s - 3ms/step - accuracy: 0.6238 - loss: 0.8479
Epoch 10/100
20/20 - 0s - 3ms/step - accuracy: 0.6479 - loss: 0.8218
Epoch 11/100
20/20 - 0s - 3ms/step - accuracy: 0.6640 - loss: 0.7981
Epoch 12/100
20/20 - 0s - 2ms/step - accuracy: 0.6688 - loss: 0.7785
Epoch 13/100
20/20 - 0s - 3ms/step - accuracy: 0.6768 - loss: 0.7608
Epoch 14/100
20/20 - 0s - 3ms/step - accuracy: 0.6897 - loss: 0.7450
Epoch 15/100
20/20 - 0s - 4ms/step - accur

In [58]:

model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

20/20 - 0s - 12ms/step - accuracy: 0.8376 - loss: 0.4737
4/4 - 0s - 8ms/step - accuracy: 0.8364 - loss: 0.5361
Train / Test Accuracy: 83.8% / 83.6%


In [13]:
from sklearn.metrics import confusion_matrix
import numpy as np

unique, counts = np.unique(y_test, return_counts=True)
counts

array([13, 22, 75], dtype=int64)

In [ ]:
'''
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
  model = tf.keras.models.load_model("SANN.keras")
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
  model.fit(x_train, y_train, epochs=100)

  y_pred = model.predict(x_test)
  class_preds = np.argmax(y_pred, axis=-1)
  matrices.append(confusion_matrix(y_test, class_preds))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix
'''

